<a href="https://colab.research.google.com/github/taguch1s/study/blob/main/pytorch_advanced/7_nlp_sentiment_transformers/7_nlp_sentiment_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 7.1 単語分割（Janome、MeCab+NEologd）

1.1 janome

In [1]:
!pip install janome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 31.5 MB/s eta 0:00:00


In [2]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)

機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [4]:
# 単語分割する関数を定義


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))

['機械', '学習', 'が', '好き', 'です', '。']


1.2 Mecab


In [5]:
!apt install mecab libmecab-dev mecab-ipadic-utf8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-14build9 [306 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-utils amd64 0.996-14build9 [4,850 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-ipadic all 2.7.0-20070801+main-3 [6,718 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mecab amd64 0.996-14build9 [136 kB]
Get:6 http://archive.ubuntu.co

In [6]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Receiving objects: 100% (75/75), 58.09 MiB | 37.31 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [7]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.6 MB/s eta 0:00:00


In [10]:
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [11]:
# 辞書のディレクトリを確認
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [12]:
import MeCab

m_t = MeCab.Tagger('-Ochasen')

text = '機械学習が好きです。'

print(m_t.parse(text))

機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [14]:
# 単語分割する関数を定義

m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')


def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret


text = '機械学習が好きです。'
print(tokenizer_mecab(text))

['機械学習', 'が', '好き', 'です', '。']


## 7.2 torchtextでのDataset、DataLoaderの実装方法